In [3]:
import  numpy as np
import tensorflow as tf
import numpy as np
from collections import Counter
import random
import torch.nn.functional
import math
def load_data(data_path):
    data =[]
    ids =[]
    max_len =0
    law_labels =[]
    money_label =[]
    with open(data_path,'r',encoding='utf8') as f:
        for line in f:
            line = line.strip()
            lines = line.split('\t')
            ids.append(lines[0])

            sentence = lines[1].split(' ')
            tmp_len = len(sentence)
            if(tmp_len>max_len):
                max_len = tmp_len
            multi_label =[int(i)-1 for i in lines[3].split(',')]
            data.append(sentence)
            law_labels.append(multi_label)
            money_label.append(lines[2])
    return ids , data, money_label,law_labels
data_path ='./data/AI_law/sample_seg_train.txt'
ids,data,money_label,law_labels = load_data(data_path)


In [ ]:
def bulid_vocabulary(data,min_count =3):
    count = [('<UNK>', -1), ('<PAD>', -1)]
    words =[]
    [words.extend(line) for line in data]
    counter = collections.Counter(words)
    counter_list = counter.most_common()
    for word,c in counter_list:
        #记录最少出现三次的词
        if c>min_count:
            count.append((word,c))
        # 同理也可以限制最多出现的词的数目
    dict_word2index = {word:c for word,c in enumerate(count)}
    dict_index2word ={c:word for word,c in enumerate(count)}
    print("vocab size:", len(count))
    print(count[-1])
    return count, dict_word2index, dict_index2word
count, dict_word2index, dict_index2word =bulid_vocabulary(data)
